### 2019 Facebook 平台商机大数据挖掘研究记录

#### 查询地名坐标 GEOPY

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Sydney, australia")
print(location.address)
print(str(location.latitude)+", "+str(location.longitude))
point = str(location.latitude)+","+str(location.longitude)

#### 根据坐标查询商户信息 FBGRAPH API

In [ ]:
import facebook
token = "EAAFTFQ68bZAsBAAx4JOTwQxWOC7oHio5ep7ZAa6MhwA3ys9ZBL0Nd95woZCC28rREV1phfE9qCHirALQR8TlmNmGnNLyZBfiLUjngrFwe8uhGMIfjD9suSWtVC6lhDFpUcTurfZAgYq5ZBjjgXWqhK3slUn9V7r8RJgRZByO0mcHEsHOyPjt3TKMZAV01Elfph9ApDSS8Bs6nKgZDZD"
query = "phonerepair"
graph = facebook.GraphAPI(access_token=token)

places = graph.search(type='place',
                      center='-33.32167,115.63692',
                      q=query,
                      fields='name,location')
for place in places['data']:
    print('%s %s' % (place['name'].encode(),place['location'].get('country')))

### STEP1 - 模拟FB登录及搜索功能 --> 输出全部搜索结果 （变参：keywords）

In [ ]:
from selenium import webdriver 
#预备延时已备网络操作时间
from time import sleep
target = "pages"
keywords = "3c repair"

#清理历史文件
import os
os.remove("page_email.csv")
print("Files Removed!")

#关闭chrome弹窗通知
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
options.add_experimental_option('prefs',prefs)
options.add_argument("disable-infobars") # 关闭'chrome正受到自动测试软件的控制'提示
options.add_argument('--hide-scrollbars')  #隐藏滚动条, 应对一些特殊页面
options.add_argument('blink-settings=imagesEnabled=false')  #不加载图片, 提升速度
options.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败

driver = webdriver.Chrome(r"C:\Users\UNclePeach\AnacondaProjects\Social Media CRM Analysis\chromedriver.exe", options = options) 
driver.get('https://www.facebook.com/')
  
username_box = driver.find_element_by_id('email') 
username_box.send_keys("taojing@csu.fullerton.edu") 

password_box = driver.find_element_by_id('pass') 
password_box.send_keys("Lettopia@201905") 

login_box = driver.find_element_by_id('loginbutton') 
login_box.click() 

driver.get("https://www.facebook.com/search/"+target+"/?q="+keywords)
# add code to scrape data of interest here

#If you want to scrool to a page with infinite loading, like social network ones, facebook etc. 
while driver.find_element_by_tag_name('div'):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    Divs=driver.find_element_by_tag_name('div').text
    if 'End of Results' in Divs:
        print ("end of results")
        break
    else:
        continue     

### STEP2.1.1 - 导出所有关联Keywords Pages的ID及邮箱

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import traceback

#count = 0
page_likes = driver.find_elements_by_class_name('PageLikeButton')
ff = open('csvfile.csv','a')
for like in page_likes:
    url="https://www.facebook.com/"+like.get_attribute("data-profileid")+"/about/"
    #print(str(count)+"-message to: "+like.get_attribute("data-profileid"))
    try:
        page = urlopen(url)
        soup = BeautifulSoup(page,'html.parser')       
        #print(soup.find_all("a"))
        for link in soup.find_all("a"):
        #print(link.get('href'))
            linkstring=link.get("href")
            if "mailto:" in linkstring:
                #count = count + 1
                #number = str(count)
                #print(number+"-email to: "+linkstring[7:len(linkstring)])  
                mailto = linkstring[7:len(linkstring)]
                ff.write(mailto) #Give your csv text here.
                ff.write('\n')## Python will convert \n to os.linesep
                
    except:
        f=open("log.txt",'a')  
        traceback.print_exc(file=f)  
        f.flush()  
        f.close()
    continue
    
#关闭所有对象    
ff.close()
driver.quit()
print("Page emails export done!")

### STEP2.1.2 - 清理csv文件重复行（重复email地址）

In [ ]:
with open('csvfile.csv','r') as in_file, open('page_email.csv','w') as out_file:
    seen = set() # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue # skip duplicate

        seen.add(line)
        out_file.write(line)

os.remove("csvfile.csv")        
print("Page email clean up!")

### STEP2.2 - 导出所有关联Keywords Pages的商家名称，地址及电话

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import traceback

count = 0
page_likes = driver.find_elements_by_class_name('PageLikeButton')
for like in page_likes:
    url="https://www.facebook.com/"+like.get_attribute("data-profileid")+"/about/"
    count = count + 1
    print(str(count)+" "+url) 
    page = urlopen(url)
    soup = BeautifulSoup(page,'html.parser')
    try:
        title = soup.title.string
        length = len(title)
        end = length - 19
        #截取店铺名
        print(title[0:end])
        #address block
        spans = soup.find_all('span', {'class':"_2iem"})
        #call block
        calls = soup.find_all('div', {"class":"_50f4"})
        if spans[0] in spans:
            print("Addr: " + str(spans[0].string)+", "+str(spans[1].string))
        if calls[0] in calls:    
            print(calls[0].string)
    except:
        f=open("log.txt",'a')  
        traceback.print_exc(file=f)  
        f.flush()  
        f.close()
    continue

### STEP3.1 - 导出所有关联Keywords Marketplace的Seller ID

In [ ]:
###改用list img.click操作收集数据

### Final - 重要操作，关闭对象

In [ ]:
#完成所有操作，关闭对象
driver.quit()